In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import multiprocessing as mp

import pickle
import joblib

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests

In [2]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [ ]:
# Load systems
esi_op = app.op['get_universe_systems']()
response = esi_client.request(esi_op);
system_ids = response.data;

In [ ]:
# Load data for all systems
ops = [app.op['get_universe_systems_system_id'](system_id = system_id) for system_id in system_ids];
responses = esi_client.multi_request(tqdm(ops));
systems = {k : resp[1].data for k,resp in zip(system_ids, responses)};

In [ ]:
import json

# Save graph
with open('universe/systems.txt', 'w') as f:
    json.dump(systems, f, indent = 4)

In [ ]:
# Loading stargates
stargate_ids = [];

for system in systems.values():
    if 'stargates' not in system:
        continue;
        
    stargate_ids.extend(system.stargates);

ops = [app.op['get_universe_stargates_stargate_id'](stargate_id = stargate_id) for stargate_id in stargate_ids];
responses = esi_client.multi_request(tqdm(ops));
stargates = {k : resp[1].data for k,resp in zip(stargate_ids, responses)};

In [ ]:
import json

# Save graph
with open('universe/stargates.txt', 'w') as f:
    json.dump(stargates, f, indent = 4)

In [ ]:
def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);

systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

nx.write_adjlist(systems_graph, 'universe/sysgraph.adjlist')

In [3]:
# Load items
pages = esi_client.request(
    app.op['get_universe_types']()
).header['X-pages'][0]

type_ids = [
    i
    for page in trange(1, pages + 1)
    for i in esi_client.request(
        app.op['get_universe_types'](page = page)
    ).data
]

In [7]:
p = mp.pool.ThreadPool(10)

def load_type(type_id):
    return esi_client.request(
        app.op['get_universe_types_type_id'](type_id = type_id)
    ).data

types = {
    tid : t
    for tid, t in zip(tqdm(type_ids), p.imap(load_type, type_ids))
}

In [5]:
import json

# Save graph
with open('universe/types.txt', 'w') as f:
    json.dump(types, f, indent = 4)

In [8]:
market_types = {
    k : v
    for k,v in types.items()
    if 'market_group_id' in v
}

In [10]:
import json

# Save graph
with open('universe/market_types.txt', 'w') as f:
    json.dump(market_types, f, indent = 4)

In [9]:
len(market_types)

14466

In [12]:
market_group_ids = esi_client.request(
    app.op['get_markets_groups']()
).data

In [13]:
len(market_group_ids)

1876